## Tauchen_MATLAB

In [9]:
import numpy as np
from scipy.stats import norm

In [10]:
# Example usage
Sigma_example = np.array([[0.1, 0], [0, 0.1]])  # Covariance matrix
B_example = np.array([[0.9, 0.05], [0.05, 0.9]])  # Lag operator matrix
ny_example = 5  # Number of grid points

In [11]:
def tauchen_var(Sigma, B, ny):
    """
    Compute the Markov transition matrix (P) and the grid (G) to match MATLAB results exactly.

    Parameters:
        Sigma (numpy.ndarray): Covariance matrix of shocks (MxM).
        B (numpy.ndarray): Lag operator matrix (MxM).
        ny (int): Number of grid points.

    Returns:
        P (numpy.ndarray): Markov transition matrix (NxN).
        G (numpy.ndarray): Grid (NxM).
    """
    # Dimensions
    M = B.shape[0]  # Number of random variables
    nSig = 3  # Fixed grid spacing

    # Compute unconditional variance
    Sigy = np.linalg.solve(np.eye(M) - np.dot(B.T, B), Sigma)
    stdy = np.sqrt(np.diag(Sigy))

    # Create grid points
    grids = [np.linspace(-nSig * stdy[m], nSig * stdy[m], ny) for m in range(M)]
    G = np.array(np.meshgrid(*grids, indexing="ij")).reshape(M, -1).T

    # Create bounds
    edges = [
        np.hstack([-np.inf, 0.5 * (grids[m][1:] + grids[m][:-1]), np.inf]) for m in range(M)
    ]
    lower_bounds = np.array(np.meshgrid(*[edges[m][:-1] for m in range(M)], indexing="ij")).reshape(M, -1).T
    upper_bounds = np.array(np.meshgrid(*[edges[m][1:] for m in range(M)], indexing="ij")).reshape(M, -1).T

    # Transition matrix
    N = G.shape[0]
    P = np.zeros((N, N))
    mu = np.dot(G, B.T)
    stde = np.sqrt(np.diag(Sigma))

    for i in range(N):
        for j in range(N):
            prob = 1
            for m in range(M):
                lower = (lower_bounds[j, m] - mu[i, m]) / stde[m]
                upper = (upper_bounds[j, m] - mu[i, m]) / stde[m]
                prob *= max(0, norm.cdf(upper) - norm.cdf(lower))
            P[i, j] = prob

    # Normalize probabilities
    P /= P.sum(axis=1, keepdims=True)

    return P, G


# Compute the Markov transition matrix and grid
P, G = tauchen_var(Sigma_example, B_example, ny_example)

In [12]:
# Display results
print("Transition Matrix (P):")
print(P)
print("\nGrid (G):")
print(G)


Transition Matrix (P):
[[8.89039002e-01 5.38496305e-02 1.52501482e-08 0.00000000e+00
  0.00000000e+00 5.38496305e-02 3.26170472e-03 9.23710707e-10
  0.00000000e+00 0.00000000e+00 1.52501482e-08 9.23710707e-10
  2.61593720e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.21523225e-02 8.72218344e-01 2.21523211e-02 1.44774450e-09
  0.00000000e+00 2.01763470e-03 7.94416924e-02 2.01763456e-03
  1.31860645e-10 0.00000000e+00 1.18172416e-09 4.65288228e-08
  1.18172408e-09 7.72304869e-17 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [1.42639899e-08 5.03674178e-02 8.23688955e-01 7.85990556e-03
  1.16857639e-10 1.90986695e-09 6.74391020e-03 1.10287257e-01
  1.05239656e-03 1.56465719e-11 2.30169949e-15 8.12750575e-09
  1.3291403